<h1 align="center"> <span style="font-size: 20px;"> Proyecto Final Grupo Nº 7 </span> <br>
    <align="center"> <span style="font-size: 15px;">  Proyecto Yelp & Google Maps - Reviewa y Recomendaciones</span><br>         
<align="center"> <span style="font-size: 15px;">@utores:  Javier Castro, Luca Ramallo, Luis Ramírez, Lesmen García.</span> <br>
  <a href="https://github.com/lesmengp/Proyecto-Final-Grupo-07.git">GitHub: <span style="font-size: 20px;">Proyecto Yelp & Google Maps</span></a> 
</h1>

<h1 align="center">Análisis de sentimientos utilizando NLTK (Natural Language Toolkit) Vader
    
### Bibliografia: https://www.nltk.org/howto/sentiment.html
    
    
- Negatividad: Siendo 0 cuando no es negativo y 1 cuando es completamente negativo<br>
- Neutralidad: El cual será 0 cuando no es neutral y 1 cuando es completamente neutral<br>
- Positividad: Que será 0 cuando no es positivo y 1 cuando es completamente positivo<br>
- Compuesto: Será -1 cuando el comentario sea completamente negativo, 0 cuando sea completamente neutro y 1 cuando sea totalmente positivo.    

### Se importan las librerias

In [1]:
from google.cloud import bigquery
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('subjectivity')

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package subjectivity to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package subjectivity is already up-to-date!


### Se obtienen los datos para analizar, seleccionan solo los que tienen 'Restaurant' y ' Food' en su categoria.

In [2]:
client = bigquery.Client()
sql = """
      SELECT yr.review_id, yr.stars, yr.useful, yr.cool, yr.text, split(yb.categories,',') AS categories 
        FROM `eternal-empire-399016.gmy_bq.df_yelp_reviews` AS yr
        INNER JOIN `eternal-empire-399016.gmy_bq.df_yelp_business` AS yb 
          ON yr.business_id = yb.business_id
        WHERE SEARCH(yb.categories, 'Restaurant') OR SEARCH(yb.categories, 'Food') 
        LIMIT 10
      """
df_yelp_reviews_business = client.query(sql).to_dataframe()

- - -
### Usando Vader

In [12]:
df_yelp_reviews_business.head(10).T

,0,1,2,3,4,5,6,7,8,9
review_id,cwip_yIbMB2OTn0ppL70FQ,50e3Ak5AfpkdOFnzruXG2w,7X0lqms3YT0elKszvx0Lzw,yMu07shy8E5M9CvbH2415Q,DfPQpFf2CCB0bzZ-eTdCHw,04o87iWioMFixD7rKeSETQ,XEiyGWpHtVnhiqjB7NgjbQ,Ga6-2aYcyyfqbmVg0gqMpg,Q3PhyVaGTnZa2LN8lWK8gg,ebpblcrsqRC3RUKm3esO9A
stars,1,1,1,1,1,1,1,1,1,1
useful,11,8,11,10,15,7,9,8,7,0
cool,0,0,7,0,1,0,1,0,0,0
text,I chose this place because it was one of the f...,I purchased a cooking class as a Christmas gif...,Frozen Mango & Coconut drink excellent. Hoste...,"Worst food ever. Workers and owners are rude, ...",My wife and I sat through the 3hr pitch. The g...,Wow. What a difference between our first visit...,"As the previous reviews stated, I wish there w...",Came here and got two Sundays. Everything was ...,I got a flat on the way to NY from MD so I put...,0 stars！Service bad AF! Go eat something else ...
categories,"[New Mexican Cuisine, Tex-Mex, Food Delivery...","[Food, Event Planning & Services, Arts & Cra...","[Food, Active Life, Swimming Pools, Restaur...","[Salad, Sandwiches, Food Delivery Services, ...","[Grocery, Food, Food Delivery Services]","[Chicken Wings, Specialty Food, Sandwiches, ...","[Italian, Restaurants, Beer, Wine & Spirits...","[Food, Ice Cream & Frozen Yogurt]","[Mobile Phones, Shopping, Food, Drugstores,...","[Food, Coffee & Tea, Fast Food, Burgers, R..."


- - -
Trabajando con los atributos "categories"<br>
No afecta al analiss de sentimiento, solo para pruebas<br>

In [13]:
df_yelp_reviews_business.categories[0][2]

' Food Delivery Services'

In [14]:
df_yelp_reviews_business.shape

(1000, 6)

In [9]:
newdf = df_yelp_reviews_business.explode('categories')

In [14]:
newdf.head()

,review_id,stars,useful,cool,text,categories
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Sandwiches
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Burgers
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Hot Dogs
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Restaurants
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Fast Food


In [11]:
# new data frame with split value columns
newdf["categories"].str.split(",", n = -1, expand = True)
newdf.reset_index(drop=True)

,review_id,stars,useful,cool,text,categories
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Sandwiches
1,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Burgers
2,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Hot Dogs
3,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Restaurants
4,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Fast Food
...,...,...,...,...,...,...
5370500,-Ax9oPBgjfKZ9Feq1j-Yvw,5,0,0,A great local place. I just discovered this l...,Local Flavor
5370501,-Ax9oPBgjfKZ9Feq1j-Yvw,5,0,0,A great local place. I just discovered this l...,Restaurants
5370502,-Ax9oPBgjfKZ9Feq1j-Yvw,5,0,0,A great local place. I just discovered this l...,Cafes
5370503,-Ax9oPBgjfKZ9Feq1j-Yvw,5,0,0,A great local place. I just discovered this l...,Food


In [12]:
newdf

,review_id,stars,useful,cool,text,categories
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Sandwiches
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Burgers
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Hot Dogs
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Restaurants
0,LOxhJVtAcNCTkwNsbCzHjw,1,0,0,The chicken is the worst I have ever tasted! I...,Fast Food
...,...,...,...,...,...,...
831111,-Ax9oPBgjfKZ9Feq1j-Yvw,5,0,0,A great local place. I just discovered this l...,Local Flavor
831111,-Ax9oPBgjfKZ9Feq1j-Yvw,5,0,0,A great local place. I just discovered this l...,Restaurants
831111,-Ax9oPBgjfKZ9Feq1j-Yvw,5,0,0,A great local place. I just discovered this l...,Cafes
831111,-Ax9oPBgjfKZ9Feq1j-Yvw,5,0,0,A great local place. I just discovered this l...,Food


In [20]:
#newdf.groupby(['categories']).size()
#df.groupby(['continent']).apply(lambda x: x[x['Member_G20'] == 'Y' ]['GDP(trillion)'].sum())
newdf_group = newdf.groupby(['categories']).apply(lambda x: x[x['categories'].str.contains('Food')])
#planets[planets.method.str.contains('Pulsa')]
newdf_group.reset_index(drop=True)

,review_id,stars,useful,cool,text,categories
0,jNaU3XKJj-fdjok1ZKqwvQ,1,0,0,My boyfriend and i come to this restaurant fai...,Comfort Food
1,uNO0E0RGC9f5m8V9WRPh3g,1,0,0,Would have loved to try this place because it ...,Comfort Food
2,ij1qQTQ5mf_i_moS5GdXPQ,1,0,0,I've always heard great things about The Eagle...,Comfort Food
3,z9uIw22ku4mMA-a7gLINBQ,1,1,0,Bottom line DO NOT EAT HERE! I recently ate he...,Comfort Food
4,ET7_uTovcArBbcQ9tNympg,1,0,0,I live right around the corner from this spot ...,Comfort Food
...,...,...,...,...,...,...
1069063,uY7GJQUx4t_ZkAi_rW17Bg,4,1,0,"There isn't much to complain about this meal, ...",Specialty Food
1069064,45WXT9EGLBd0gnbNlv10kw,2,44,35,"Maido! has seen better days.\n\nThe ""Marketpla...",Specialty Food
1069065,3L-0urSAB4TqOFVi72Dy7Q,5,3,1,"Superb!! 5 stars for greeting, friendliness, k...",Specialty Food
1069066,XKo4EiRpz5j9qmYuTr_B8g,1,0,0,We came in tonight to celebrate our 22nd birth...,Specialty Food


In [22]:
newdf_group.groupby(['review_id']).size()

review_id
---wmv2Keu09L06rcXlAdg    2
--03GwblmsiubcsmY4qVLw    1
--0C2F8aaAc6a96d9_Kj2g    1
--0ecRcxmpWBGLW4WO529w    1
--1AeaJVhpmvZDII85Tzig    1
                         ..
zzyCM-bvT8pRjCH2UInXLw    1
zzyDEHIj6NO5cZzCE2x4nQ    1
zzyc3-6g3r-Vy2LNwO7m1Q    1
zzydBrFJXdNKZwtluYyuMA    1
zzyuJYKUPMD6u_NmmFmZJQ    1
Length: 830801, dtype: int64

In [195]:
lista_categorias_eliminar = [" Acai Bowls"," Accessories"," Accountants"," Active Life"," Acupuncture",
                             " Adult", " Adult Education"," Adult Entertainment"," Afghan"," African",
                             " Airport Shuttles"," Airports"," Alternative Medicine"," Art Museums",
                             " Animal Shelters"," Antiques"," Apartments"," Appliances"," Appliances & Repair",
                             " Architects"," Arcades"," Art Classes"," Art Galleries"," Art Supplies",
                             " Arts & Crafts"," Arts & Entertainment"]

newdf.drop(newdf[newdf['categories'].isin(lista_categorias_eliminar)].index, axis=0, inplace=True)
#df = newdf[~newdf.categories.isin(lista_categorias_eliminar)]
#newdf.drop(newdf[newdf.categories == " Acai Bowls"].index, axis=0, inplace=True)

In [196]:
newdf.groupby(['categories']).size().T

categories
 Arabic                   397
 Argentine                494
 Armenian                 190
 Asian Fusion           16005
 Australian               443
                        ...  
Wine Bars                 586
Wine Tasting Classes       26
Wine Tours                 17
Women's Clothing            4
Wraps                     255
Length: 864, dtype: int64

Fin trabajando con los atributos de "categories"<br>
- - -

In [9]:
# neg: Siendo 0 cuando no es negativo y 1 cuando es completamente negativo
# neu: El cual será 0 cuando no es neutral y 1 cuando es completamente neutral
# pos: Que será 0 cuando no es positivo y 1 cuando es completamente positivo
# compound: Será -1 cuando el comentario sea completamente negativo, 0 cuando sea completamente neutro y 1 cuando sea totalmente positivo.
# Ejemplo usando compound: 
# para valores mayor a 10% (0.1) se puede considerar positivo,
# para valores menor a 10% (-0.1) se puede considerar negativo,
# para valores mayor a negativo y menor a positivo se puede considerar neutro,
def analisis_sentimento(text:str,positivo:int,negativo:int):
    lines_list = tokenize.sent_tokenize(text)
    sentimiento = 0
    Compuesto = 0
    #lines_list.extend(lines_list)
    for sentence in lines_list:
        sid = SentimentIntensityAnalyzer()
        ss = sid.polarity_scores(sentence)
        Compuesto += ss['compound']
    if Compuesto/len(lines_list) >= (positivo/100):
        sentimiento = 1
    elif Compuesto/len(lines_list) <= -(negativo/100):
        sentimiento = -1
    return(sentimiento)

In [10]:
texto = df_yelp_reviews_business.iloc[4,4]
print(texto)
analisis_sentimento(texto,10,10)

Been here many times and my order is messed up every time. Tried giving them the benefitt of the doubt but I think its just ineptitude. Poor or nonexistant manners as well.


-1

In [24]:
def analisis_sentimento_general(text:str):
    lines_list = tokenize.sent_tokenize(text)
    resultado_analisis = {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}
    #lines_list.extend(lines_list)
    for sentence in lines_list:
        sid = SentimentIntensityAnalyzer()
        print(sentence)
        ss = sid.polarity_scores(sentence)
        for k in sorted(ss):
            print('{0}: {1}, '.format(k, ss[k]), end='')
        print()

In [28]:
analisis_sentimento_general(texto)

If Yelp allowed negative stars as ratings, Founding Farmers would surely rate as such.
compound: -0.2023, neg: 0.199, neu: 0.645, pos: 0.156, 
I gave this establishment 3 tries since I had a good experience at the DC location.
compound: 0.4404, neg: 0.0, neu: 0.791, pos: 0.209, 
The first time was during opening weekend and the service was a little spotty, but expected.
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, 
We started with the toast with cheese and shallots - there was entirely too much cheese and the shallots were overwhelming.
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, 
I got a pasta and it was underwhelming, which was kind of expected as it's not a pasta place.
compound: 0.0, neg: 0.0, neu: 1.0, pos: 0.0, 
My husband got the farmers plate and it was just okay.
compound: 0.2263, neg: 0.0, neu: 0.84, pos: 0.16, 
I returned again the next morning for brunch with girlfriends and again, service was not good.
compound: -0.3412, neg: 0.147, neu: 0.853, pos: 0.0, 
The menu for bre

### Verificando todos los sentimientos de las reseñas.

In [52]:
for i,fila in df_yelp_reviews_business.iterrows():
    #print(i, " -> ",analisis_sentimento_wrk_2(fila['text']))
    sentimiento_es = lambda x : "positivo" if (x == 1) else ("Negativo" if (x == -1) else "Neutro")
    print("La reseña: ", i, " es: ", sentimiento_es(analisis_sentimento(fila['text'],10,10)))

La reseña:  0  es:  Negativo
La reseña:  1  es:  Negativo
La reseña:  2  es:  positivo
La reseña:  3  es:  Negativo
La reseña:  4  es:  Neutro
La reseña:  5  es:  Neutro
La reseña:  6  es:  Negativo
La reseña:  7  es:  Negativo
La reseña:  8  es:  Negativo
La reseña:  9  es:  Neutro


In [96]:
### Creando una columna 
import datetime

date_format_str="%Y-%m-%d %H:%M:%S.%f"

lista_sentimientos = []

#print("Hora Inicio: ",datetime.datetime.now())
for i,fila in df_yelp_reviews_business.iterrows():
    lista_sentimientos.append(analisis_sentimento(fila['text'],10,10))
    #print(sentimiento)
#print(lista_sentimientos)
df_sentimientos = pd.DataFrame(lista_sentimientos)
df_yelp_reviews_business = pd.concat([df_sentimientos])

KeyError: 'text'

In [97]:
df_yelp_reviews_business

,0
0,-1
1,-1
2,1
3,-1
4,0
5,0
6,-1
7,-1
8,-1
9,0
